In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import cv2
import os
from PIL import Image, ImageChops
import numpy as np
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow

In [ ]:
def trim(im):
    bg = Image.new(im.mode, im.size, im.getpixel((0,0)))
    diff = ImageChops.difference(im, bg)
    diff = ImageChops.add(diff, diff, 2.0, -100)
    bbox = diff.getbbox()
    if bbox:
        return im.crop(bbox)

def auto_canny(image, sigma=0.33):
	v = np.median(image)
	lower = int(max(0, (1.0 - sigma) * v))
	upper = int(min(255, (1.0 + sigma) * v))
	edged = cv2.Canny(image, lower, upper)

	return edged

In [172]:
def detect_tumor_pos(path):
  raw_img = Image.open(path)
  img = np.array(trim(raw_img))
  img_scaled = cv2.resize(img, (512, 512))
  img_shifted = cv2.pyrMeanShiftFiltering(img_scaled, 21, 51)
  img_gray = cv2.cvtColor(img_shifted, cv2.COLOR_BGR2GRAY, 0.7)
  (T, thresh) = cv2.threshold(img_gray, 95, 255, cv2.THRESH_BINARY)
  kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (10, 5))
  closed = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
  closed = cv2.erode(closed, None, iterations = 7)
  closed = cv2.dilate(closed, None, iterations = 9)
  canny = auto_canny(closed)
  image_center = np.array((int(img_scaled.shape[0] / 2), int(img_scaled.shape[1] / 2)))
  (cnts, j) = cv2.findContours(canny.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
  cnts_centers = []
  for c in cnts:
    cnts_centers.append(tuple(c.mean(axis=0)))
  cnts_centers = np.array(cnts_centers)
  distances = np.linalg.norm(cnts_centers - image_center, axis=2)
  min_index = np.argmin(distances)
  cv2.drawContours(img_scaled, [cnts[min_index]], -1, (0, 0, 255), 2)
  cv2_imshow(img_scaled)

In [175]:
detect_tumor_pos('/content/drive/MyDrive/Machine Learning Assignment/Dataset/Training/meningioma/Tr-meTr_0001.jpg')
detect_tumor_pos('/content/drive/MyDrive/Machine Learning Assignment/Dataset/Training/meningioma/Tr-meTr_0002.jpg')
detect_tumor_pos('/content/drive/MyDrive/Machine Learning Assignment/Dataset/Training/meningioma/Tr-meTr_0003.jpg')
detect_tumor_pos('/content/drive/MyDrive/Machine Learning Assignment/Dataset/Training/meningioma/Tr-meTr_0004.jpg')
detect_tumor_pos('/content/drive/MyDrive/Machine Learning Assignment/Dataset/Training/meningioma/Tr-meTr_0005.jpg')
detect_tumor_pos('/content/drive/MyDrive/Machine Learning Assignment/Dataset/Training/meningioma/Tr-meTr_0006.jpg')
detect_tumor_pos('/content/drive/MyDrive/Machine Learning Assignment/Dataset/Training/glioma/Tr-glTr_0009.jpg')
detect_tumor_pos('/content/drive/MyDrive/Machine Learning Assignment/Dataset/Training/meningioma/Tr-meTr_0000.jpg')
detect_tumor_pos('/content/drive/MyDrive/Machine Learning Assignment/Dataset/Training/meningioma/Tr-meTr_0002.jpg')
detect_tumor_pos('/content/drive/MyDrive/Machine Learning Assignment/Dataset/Training/pituitary/Tr-piTr_0009.jpg')
detect_tumor_pos('/content/drive/MyDrive/Machine Learning Assignment/Dataset/Training/pituitary/Tr-pi_0048.jpg')
detect_tumor_pos('/content/drive/MyDrive/Machine Learning Assignment/Dataset/Training/pituitary/Tr-pi_0079.jpg')

Output hidden; open in https://colab.research.google.com to view.